In [85]:
import pandas as pd
import numpy as np
import scipy as sp
import math
import sympy as sym
import time

In [148]:
priorsP = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['dist. shape','uni-low','uni-up','log-uni-low','log-uni-up','norm-cen','norm-spread','log-norm-cen','log-norm-spread'])
paramsdf = pd.DataFrame(columns = ['name','mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['theKBO','obj1','obj2'])

paramsdf['name'] = ['Haumea','Hi\'aka','Namaka']
paramsdf['mass'] = [4.006*10**21,1.79*10*19,1.79*10**18]
paramsdf['sma'] = [np.NaN,49880,25657]
paramsdf['ecc'] = [np.NaN,0.0513,0.249]
paramsdf['aop'] = [np.NaN,154.1,178.9]
paramsdf['inc'] = [np.NaN,126.356,113.013]
paramsdf['lan'] = [np.NaN,206.766,205.016]
paramsdf['mea'] = [np.NaN,152.8,178.5]
paramsdf['j2r2'] = [np.NaN,4,5]
paramsdf['c22r2'] = [np.NaN,4,6]
paramsdf['spaop'] = [np.NaN,4,5]
paramsdf['spinc'] = [np.NaN,2,1]
paramsdf['splan'] = [np.NaN,4,6]

priorsP['mass'] = [2,0,10,1,2,1,1,4.006*10**21,0.040*10*20]
priorsP['sma'] = [2,0,12,1,2,35000,20000,1,1]
priorsP['ecc'] = [2,0,1,0,1,1,1,1,1]
priorsP['aop'] = [2,0,180,1,2,1,1,1,1]
priorsP['inc'] = [2,0,180,1,2,1,1,1,1]
priorsP['lan'] = [2,0,360,0,12,1,1,1,1]
priorsP['mea'] = [2,0,180,1,2,1,1,1,1]
priorsP['j2r2'] = [2,0,12,1,2,1,1,1,1]
priorsP['c22r2'] = [2,0,12,1,8,1,1,1,1]
priorsP['spaop'] = [2,0,16,1,6,1,1,1,1]
priorsP['spinc'] = [2,0,12,0,8,1,1,1,1]
priorsP['splan'] = [2,0,12,1,8,1,1,1,1]

priors1 = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['dist. shape','uni-low','uni-up','log-uni-low','log-uni-up','norm-cen','norm-spread','log-norm-cen','log-norm-spread'])
priors2 = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['dist. shape','uni-low','uni-up','log-uni-low','log-uni-up','norm-cen','norm-spread','log-norm-cen','log-norm-spread'])

priors1['mass'] = [3,0,10,1,2,1,1,1.79*10**19,0.11*10**19]
priors1['sma'] = [2,0,12,1,2,49880,198,1,1]
priors1['ecc'] = [2,0,1,0,1,1,1,1,1]
priors1['aop'] = [2,0,180,1,2,1,1,1,1]
priors1['inc'] = [2,0,180,1,2,1,1,1,1]
priors1['lan'] = [2,0,360,0,12,1,1,1,1]
priors1['mea'] = [2,0,180,1,2,1,1,1,1]
priors1['j2r2'] = [2,0,12,1,2,1,1,1,1]
priors1['c22r2'] = [2,0,12,1,8,1,1,1,1]
priors1['spaop'] = [2,0,16,1,6,1,1,1,1]
priors1['spinc'] = [2,0,12,0,8,1,1,1,1]
priors1['splan'] = [2,0,12,1,8,1,1,1,1]

priors2['mass'] = [2,0,10,1,2,1,1,1.79*10**18,1.48*10**18]
priors2['sma'] = [2,0,12,1,2,25657,91,1,1]
priors2['ecc'] = [2,0,1,0,1,1,1,1,1]
priors2['aop'] = [2,0,180,1,2,1,1,1,1]
priors2['inc'] = [2,0,180,1,2,1,1,1,1]
priors2['lan'] = [2,0,360,0,12,1,1,1,1]
priors2['mea'] = [2,0,180,1,2,1,1,1,1]
priors2['j2r2'] = [2,0,12,1,2,1,1,1,1]
priors2['c22r2'] = [2,0,12,1,8,1,1,1,1]
priors2['spaop'] = [2,0,16,1,6,1,1,1,1]
priors2['spinc'] = [2,0,12,0,8,1,1,1,1]
priors2['splan'] = [2,0,12,1,8,1,1,1,1]

In [151]:
def Not_mm_priors(priors, params):
    columnList = list(priors)
    totalLogProb = 0
    
    probDist = pd.DataFrame(columns = ['mass','sma','ecc','aop','inc','lan','mea','j2r2','c22r2','spaop','spinc','splan'],index=['PDF'])
   
    paramsList = params.to_numpy().tolist()
    count = 0
    allProbs = []
    numNaNs = 0
    #This loop runs through every column in the priors dataframe, and evaluates the probability density
    #function of the specified type.
    for i in columnList:
        count += 1
        
        #Uniform Distribution Shape
        if priors[i][0] == 0:
            for x in paramsList:
                if x[count] < priors[i][2] and x[count] > priors[i][1]:
                    allProbs.append(1)
                elif np.isnan(x[count]):
                    numNaNs += 1
                else:
                    allProbs.append(0)
        
        #Log-Uniform Distribution Shape
        elif priors[i][0] == 1:
            for x in paramsList:
                if x[count] < priors[i][4] and x[count] > priors[i][3]:
                    allProbs.append(1)
                elif np.isnan(x[count]):
                    numNaNs += 1
                else:
                    allProbs.append(0)
            
        # Normal Distribution Shape
        elif priors[i][0] == 2:
            for x in paramsList:
                if not np.isnan(x[count]):
                    allProbs.append(np.exp(-1/2*((x[count]-priors[i][6])/priors[i][5])**2))
            
        #Log Normal Distribution Shape
        elif priors[i][0] == 3:
            for x in paramsList:
                if not np.isnan(x[count]):
                    allProbs.append(np.exp(-1/2*(((np.log(x[count])-priors[i][8])**2)/(priors[i][7])**2))/x[count])
        else:
            print('Invalid input.') 
            
        #Here, add the Prior Probability Density function for this element to the total
    for x in allProbs:
        totalLogProb = totalLogProb + np.log(x)
        print(x)
        
    return totalLogProb

In [153]:
dist = Not_mm_priors(priorsP, paramsdf)
print('Total Log Probability: ', dist)


0.0
0.0
0.0
0.69460354097969
0.9870230388205508
0.6376180484837531
0.7542733073963409
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.011108996538242306
0.00033546262790251185
0.011108996538242306
3.726653172078671e-06
0.011108996538242306
0.00033546262790251185
0.6065306597126334
1.0
0.011108996538242306
3.726653172078671e-06
Total Log Probability:  -inf


C:\Users\annas\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: RuntimeWarning: divide by zero encountered in log


In [136]:
priorsDF = pd.DataFrame(columns = ['mass_1','mass_2','sma_2','ecc_2','aop_2','inc_2','lan_2','mea_2','mass_3','sma_3','ecc_3','aop_3','inc_3','lan_3','mea_3','j2r2_1','c22r2_1','spaop_1','spinc_1','splan_1','sprate_1'],index=['dist. shape','uni-low','uni-up','log-uni-low','log-uni-up','norm-cen','norm-spread','log-norm-cen','log-norm-spread'])
allParamsDF = pd.DataFrame(columns = ['name_1','mass_1','name_2','mass_2','sma_2','ecc_2','aop_2','inc_2','lan_2','mea_2','name_3','mass_3','sma_3','ecc_3','aop_3','inc_3','lan_3','mea_3','j2r2_1','c22r2_1','spaop_1','spinc_1','splan_1','sprate_1'],index=[0])

#Set Parameters, or known data of objects
allParamsDF['name_1'][0] = 'Haumea'
allParamsDF['name_2'][0] = 'Hi\'aka'
allParamsDF['name_3'][0] = 'Namaka'

allParamsDF['mass_1'][0] = 4.006*10**21
allParamsDF['mass_2'][0] = 1.79*10**19
allParamsDF['mass_3'][0] = 1.79*10*18

allParamsDF['sma_2'][0] = 49880
allParamsDF['sma_3'][0] = 25657

allParamsDF['ecc_2'][0] = 0.0513
allParamsDF['ecc_3'][0] = 0.249

allParamsDF['aop_2'][0] = 154.1
allParamsDF['aop_3'][0] = 178.9

allParamsDF['inc_2'][0] = 126.356
allParamsDF['inc_3'][0] = 113.013

allParamsDF['lan_2'][0] = 206.766
allParamsDF['lan_3'][0] = 205.016

allParamsDF['mea_2'][0] = 152.8
allParamsDF['mea_3'][0] = 178.5

allParamsDF['j2r2_1'][0] = 1
allParamsDF['c22r2_1'][0] = 12
allParamsDF['spaop_1'][0] = 'idk'
allParamsDF['spinc_1'][0] = 'irdk'
allParamsDF['splan_1'][0] = 'isdk'
allParamsDF['sprate_1'][0] = 'wotfam'


#Set Priors
priorsDF['mass_1'][0] = 2
priorsDF['mass_1'][1] = 0
priorsDF['mass_1'][2] = 0
priorsDF['mass_1'][3] = 0
priorsDF['mass_1'][4] = 0
priorsDF['mass_1'][5] = 4.006*10**21
priorsDF['mass_1'][6] = 0.0040*10**21
priorsDF['mass_1'][7] = 4.006*10**21
priorsDF['mass_1'][8] = 0.0040*10**21

priorsDF['mass_2'][0] = 2
priorsDF['mass_2'][1] = 0
priorsDF['mass_2'][2] = 0
priorsDF['mass_2'][3] = 0
priorsDF['mass_2'][4] = 0
priorsDF['mass_2'][5] = 1.79*10**19
priorsDF['mass_2'][6] = 0.11*10**19
priorsDF['mass_2'][7] = 1.79*10**19
priorsDF['mass_2'][8] = 0.11*10**19

priorsDF['mass_3'][0] = 2
priorsDF['mass_3'][1] = 0
priorsDF['mass_3'][2] = 0
priorsDF['mass_3'][3] = 0
priorsDF['mass_3'][4] = 0
priorsDF['mass_3'][5] = 1.79*10**18
priorsDF['mass_3'][6] = 1.48*10**18
priorsDF['mass_3'][7] = 1.79*10**18
priorsDF['mass_3'][8] = 1.48*10**18

priorsDF['sma_2'][0] = 2
priorsDF['sma_2'][1] = 0
priorsDF['sma_2'][2] = 0
priorsDF['sma_2'][3] = 0
priorsDF['sma_2'][4] = 0
priorsDF['sma_2'][5] = 49880
priorsDF['sma_2'][6] = 198
priorsDF['sma_2'][7] = 49880
priorsDF['sma_2'][8] = 198

priorsDF['sma_3'][0] = 2
priorsDF['sma_3'][1] = 0
priorsDF['sma_3'][2] = 0
priorsDF['sma_3'][3] = 0
priorsDF['sma_3'][4] = 0
priorsDF['sma_3'][5] = 25657
priorsDF['sma_3'][6] = 91
priorsDF['sma_3'][7] = 25657
priorsDF['sma_3'][8] = 91

priorsDF['ecc_2'][0] = 2
priorsDF['ecc_2'][1] = 0
priorsDF['ecc_2'][2] = 1
priorsDF['ecc_2'][3] = 0
priorsDF['ecc_2'][4] = 1
priorsDF['ecc_2'][5] = 0.0513
priorsDF['ecc_2'][6] = 0.0078
priorsDF['ecc_2'][7] = 0.0513
priorsDF['ecc_2'][8] = 0.0078

priorsDF['ecc_3'][0] = 2
priorsDF['ecc_3'][1] = 0
priorsDF['ecc_3'][2] = 1
priorsDF['ecc_3'][3] = 0
priorsDF['ecc_3'][4] = 1
priorsDF['ecc_3'][5] = 0.249
priorsDF['ecc_3'][6] = 0.015
priorsDF['ecc_3'][7] = 0.249
priorsDF['ecc_3'][8] = 0.015

priorsDF['aop_2'][0] = 2
priorsDF['aop_2'][1] = 0
priorsDF['aop_2'][2] = 180
priorsDF['aop_2'][3] = 0
priorsDF['aop_2'][4] = 180
priorsDF['aop_2'][5] = 154.1
priorsDF['aop_2'][6] = 5.8
priorsDF['aop_2'][7] = 154.1
priorsDF['aop_2'][8] = 5.8

priorsDF['aop_3'][0] = 2
priorsDF['aop_3'][1] = 0
priorsDF['aop_3'][2] = 180
priorsDF['aop_3'][3] = 0
priorsDF['aop_3'][4] = 180
priorsDF['aop_3'][5] = 178.9
priorsDF['aop_3'][6] = 2.3
priorsDF['aop_3'][7] = 178.9
priorsDF['aop_3'][8] = 2.3

priorsDF['inc_2'][0] = 2
priorsDF['inc_2'][1] = 0
priorsDF['inc_2'][2] = 180
priorsDF['inc_2'][3] = 0
priorsDF['inc_2'][4] = 180
priorsDF['inc_2'][5] = 126.356
priorsDF['inc_2'][6] = 0.064
priorsDF['inc_2'][7] = 126.356
priorsDF['inc_2'][8] = 0.064

priorsDF['inc_3'][0] = 2
priorsDF['inc_3'][1] = 0
priorsDF['inc_3'][2] = 180
priorsDF['inc_3'][3] = 0
priorsDF['inc_3'][4] = 180
priorsDF['inc_3'][5] = 113.013
priorsDF['inc_3'][6] = 0.075
priorsDF['inc_3'][7] = 113.013
priorsDF['inc_3'][8] = 0.075

priorsDF['lan_2'][0] = 2
priorsDF['lan_2'][1] = 0
priorsDF['lan_2'][2] = 1
priorsDF['lan_2'][3] = 0
priorsDF['lan_2'][4] = 1
priorsDF['lan_2'][5] = 206.766
priorsDF['lan_2'][6] = 0.033
priorsDF['lan_2'][7] = 206.766
priorsDF['lan_2'][8] = 0.033

priorsDF['lan_3'][0] = 2
priorsDF['lan_3'][1] = 0
priorsDF['lan_3'][2] = 1
priorsDF['lan_3'][3] = 0
priorsDF['lan_3'][4] = 1
priorsDF['lan_3'][5] = 205.016
priorsDF['lan_3'][6] = 0.228
priorsDF['lan_3'][7] = 205.016
priorsDF['lan_3'][8] = 0.228

priorsDF['mea_2'][0] = 2
priorsDF['mea_2'][1] = 0
priorsDF['mea_2'][2] = 1
priorsDF['mea_2'][3] = 0
priorsDF['mea_2'][4] = 1
priorsDF['mea_2'][5] = 152.8
priorsDF['mea_2'][6] = 6.1
priorsDF['mea_2'][7] = 152.8
priorsDF['mea_2'][8] = 6.1

priorsDF['mea_3'][0] = 2
priorsDF['mea_3'][1] = 0
priorsDF['mea_3'][2] = 1
priorsDF['mea_3'][3] = 0
priorsDF['mea_3'][4] = 1
priorsDF['mea_3'][5] = 178.5
priorsDF['mea_3'][6] = 1.7
priorsDF['mea_3'][7] = 178.5
priorsDF['mea_3'][8] = 1.7

priorsDF['j2r2_1'] = 1
priorsDF['c22r2_1'] = 12
priorsDF['spaop_1'] = 'idk'
priorsDF['spinc_1'] = 'irdk'
priorsDF['splan_1'] = 'isdk'
priorsDF['sprate_1'] = 'wotfam'

print(allParamsDF.transpose(),'\n\n',priorsDF.transpose())

priorsDF.to_csv(r'Priors.csv',sep='\t',index=True)
priorsDF.transpose().to_csv(r'TransposedPriors.csv',sep='\t',index=True)

                  0
name_1       Haumea
mass_1    4.006e+21
name_2       Hi'aka
mass_2     1.79e+19
sma_2         49880
ecc_2        0.0513
aop_2         154.1
inc_2       126.356
lan_2       206.766
mea_2         152.8
name_3       Namaka
mass_3        322.2
sma_3         25657
ecc_3         0.249
aop_3         178.9
inc_3       113.013
lan_3       205.016
mea_3         178.5
j2r2_1            1
c22r2_1          12
spaop_1         idk
spinc_1        irdk
splan_1        isdk
sprate_1     wotfam 

          dist. shape uni-low  uni-up log-uni-low log-uni-up   norm-cen  \
mass_1             2       0       0           0          0  4.006e+21   
mass_2             2       0       0           0          0   1.79e+19   
sma_2              2       0       0           0          0      49880   
ecc_2              2       0       1           0          1     0.0513   
aop_2              2       0     180           0        180      154.1   
inc_2              2       0     180           0     

In [139]:
def mm_priors(priors, params):
    columnList = list(priors)
    totalLogProb = 0
    
    probDist = pd.DataFrame(columns = ['mass_1','mass_2','sma_2','ecc_2','aop_2','inc_2','lan_2','mea_2','mass_3','sma_3','ecc_3','aop_3','inc_3','lan_3','mea_3','j2r2_1','c22r2_1','spaop_1','spinc_1','splan_1','sprate_1'],index=['PDF'])
   
    count = 0
    allProbs = []
    numNaNs = 0
    #This loop runs through every column in the priors dataframe, and evaluates the probability density
    #function of the specified type.
    for i in columnList:
        count += 1
        
        #Uniform Distribution Shape
        if priors[i][0] == 0:
            if params[i][0] < priors[i][2] and params[i][0] > priors[i][1]:
                allProbs.append(1)
            elif np.isnan(x[count]):
                numNaNs += 1
            else:
                allProbs.append(0)
        
        #Log-Uniform Distribution Shape
        elif priors[i][0] == 0:
            if params[i][0] < priors[i][4] and params[i][0] > priors[i][3]:
                allProbs.append(1)
            elif np.isnan(params[i][0]):
                numNaNs += 1
            else:
                allProbs.append(0)
            
        # Normal Distribution Shape
        elif priors[i][0] == 2:
            if not np.isnan(params[i][0]):
                allProbs.append(np.exp(-1/2*((params[i][0]-priors[i][6])/priors[i][5])**2))
            
        #Log Normal Distribution Shape
        elif priors[i][0] == 3:
            if not np.isnan(params[i][0]):
                allProbs.append(np.exp(-1/2*(((np.log(params[i][0])-priors[i][8])**2)/(priors[i][7])**2))/params[i][0])
        else:
            print('Invalid input for: ', i) 
            
        #Here, add the Prior Probability Density function for this element to the total
    for x in allProbs:
        totalLogProb = totalLogProb + np.log(x)
        print(x)
        
    return totalLogProb

In [140]:
dist = mm_priors(priorsDF,allParamsDF)
print('Total Probability: 10^(',dist,')')

Invalid input for:  j2r2_1
Invalid input for:  c22r2_1
Invalid input for:  spaop_1
Invalid input for:  spinc_1
Invalid input for:  splan_1
Invalid input for:  sprate_1
0.6071362817376864
0.6437558808339235
0.608938286802129
0.6980166458512227
0.6293483196697449
0.6068378707584218
0.6066274624345537
0.630731264295467
0.7104817248522851
0.6086818877746099
0.6430238118476723
0.6143279957288144
0.606933177943272
0.6072051872294486
0.6123069671246485
Total Probability: 10^( -6.9740862131907955 )
